In [1]:
from tqdm import tqdm_notebook
import ast
import pretty_midi
import numpy as np
import datetime as dt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
def process_string(s):
    split, tempo, notes = s.split('#')
    tempo = float(tempo)
    notes = [pretty_midi.Note(start=tpl[0], end=tpl[1], pitch=tpl[2], velocity=tpl[3]) for tpl in ast.literal_eval(notes)]
    return split, tempo, notes

nb_songs = 0
with open('../data/songs.txt') as f:
    for line in f:
        nb_songs += 1

In [ ]:
train_songs = []
validation_songs = []
with open('../data/songs.txt') as f:
    for line in tqdm_notebook(f, total=nb_songs):
        split, tempo, notes = process_string(line)
        if split == 'train':
            train_songs.append(notes)
        elif split == 'validation':
            validation_songs.append(notes)

In [38]:
notes[0:10]

[Note(start=1.016667, end=1.214583, pitch=48, velocity=40),
 Note(start=1.545833, end=1.780208, pitch=52, velocity=46),
 Note(start=1.000000, end=2.061458, pitch=79, velocity=72),
 Note(start=2.060417, end=2.183333, pitch=77, velocity=59),
 Note(start=2.047917, end=2.247917, pitch=48, velocity=35),
 Note(start=2.166667, end=2.257292, pitch=76, velocity=65),
 Note(start=2.262500, end=2.346875, pitch=74, velocity=69),
 Note(start=2.379167, end=2.622917, pitch=72, velocity=63),
 Note(start=2.508333, end=2.744792, pitch=50, velocity=44),
 Note(start=2.542708, end=2.764583, pitch=79, velocity=61)]

In [99]:
def normalize_song(song, chord_time=0.05):
    
    song.sort(key=lambda note: note.start)
    
    # Songs all start at t=0
    first_note_start = min(note.start for note in song)
    last_note_time = -1  # every song begins after 1 second delay on a out-of-range note
    for note in song:
        note.start -= first_note_start
        note.end -= first_note_start
        assert note.start >= last_note_time  # check that notes are sorted

        if note.start <= last_note_time + chord_time:  # simple algorithm to detect chords
            note.start = last_note_time
        else:
            last_note_time = note.start

    song.sort(key=lambda note: (note.start, note.pitch))
    
    return song

In [101]:
for song in train_songs + validation_songs:
    normalize_song(song)

In [103]:
validation_songs[0][0:20]

[Note(start=0.000000, end=0.117188, pitch=52, velocity=33),
 Note(start=0.000000, end=0.092448, pitch=62, velocity=30),
 Note(start=0.968750, end=1.084635, pitch=58, velocity=35),
 Note(start=1.020833, end=1.138021, pitch=80, velocity=62),
 Note(start=1.308594, end=1.355469, pitch=81, velocity=70),
 Note(start=1.479167, end=1.524740, pitch=68, velocity=67),
 Note(start=1.479167, end=1.531250, pitch=82, velocity=72),
 Note(start=1.613281, end=1.696615, pitch=49, velocity=55),
 Note(start=1.613281, end=1.707031, pitch=55, velocity=18),
 Note(start=1.613281, end=1.643229, pitch=65, velocity=51),
 Note(start=1.613281, end=1.662760, pitch=69, velocity=58),
 Note(start=1.613281, end=1.654948, pitch=83, velocity=72),
 Note(start=1.747396, end=1.789062, pitch=70, velocity=76),
 Note(start=1.747396, end=1.808594, pitch=84, velocity=79),
 Note(start=1.803385, end=1.839844, pitch=46, velocity=57),
 Note(start=1.803385, end=1.867188, pitch=52, velocity=37),
 Note(start=1.859375, end=1.917969, pitc

In [53]:
def constrain_pitch(pitch, min_pitch, max_pitch):
    max_pitch -= 1  # exclusive
    if pitch < min_pitch:
        pitch = pitch % 12 + 12 * (min_pitch // 12) + 12
        if pitch >= min_pitch + 12:
            pitch -= 12
        assert min_pitch <= pitch < min_pitch + 12
        return pitch
    if pitch > max_pitch:
        pitch = pitch % 12 + 12 * (max_pitch // 12) - 12
        if pitch <= max_pitch - 12:
            pitch += 12
        assert max_pitch - 12 < pitch <= max_pitch
        return pitch
    return pitch

In [69]:
class FeaturedNote:
    
    def __init__(self, pitch, wait, min_pitch, max_pitch):
        self.nb_pitches = max_pitch - min_pitch + 1
        if pitch is None:
            # Starting note, use special out-of-bounds value
            self.pitch = max_pitch - min_pitch
        else:
            self.pitch = constrain_pitch(pitch, min_pitch, max_pitch) - min_pitch
            assert 0 <= self.pitch < max_pitch - min_pitch
        self.wait = wait
        
    def __repr__(self):
        return 'FeaturedNote(pitch={pitch}, wait={wait})'.format(**self.__dict__)
        
    def calculate_features(self):
        pitch_features = [0] * self.nb_pitches
        pitch_features[self.pitch] = 1
        self.features = np.array(pitch_features + [self.wait])
        self.pitch_label = np.array(pitch_features)

In [70]:
nb_notes_history = 16
min_pitch = 45  # inclusive
max_pitch = 85  # exclusive
nb_pitches = max_pitch - min_pitch + 1  # including out of bounds pitch
nb_features = nb_pitches + 1  # including scalar continuous wait feature

In [71]:
def feature_notes(song, min_pitch, max_pitch, chord_time=0.05):

    first_note = FeaturedNote(pitch=None, wait=0, min_pitch=min_pitch, max_pitch=max_pitch)
    featured_notes = [first_note for i in range(nb_notes_history)]
    previous_time = -1
    for note in song:
        wait = note.start - previous_time
        fnote = FeaturedNote(note.pitch, wait, min_pitch=min_pitch, max_pitch=max_pitch)
        featured_notes.append(fnote)
        previous_time = note.start
            
    return featured_notes

In [72]:
def feature_songs(songs, min_pitch, max_pitch, nb_notes_history, chord_time=0.05):
    
    nb_datapoints = sum(len(song) for song in songs)
    
    sequences = -1 * np.ones(shape=(nb_datapoints, nb_notes_history, nb_features))
    pitch_labels = -1 * np.ones(shape=(nb_datapoints, nb_pitches))
    wait_labels = -1 * np.ones(shape=(nb_datapoints,))
    
    data_index = 0
    
    for song in tqdm_notebook(songs):
        
        featured_notes = feature_notes(song, min_pitch, max_pitch, chord_time)
        for fnote in featured_notes:
            fnote.calculate_features(min_pitch, max_pitch)
            
        for i in range(nb_notes_history, len(featured_notes)):
            sequences[data_index] = np.array([fnote.features for fnote in featured_notes[i - nb_notes_history:i]])
            pitch_labels[data_index] = featured_notes[i].pitch_label
            wait_labels[data_index] = featured_notes[i].wait
            data_index += 1
    
    return sequences, pitch_labels, wait_labels

In [73]:
feature_notes(validation_songs[0], min_pitch, max_pitch)

[FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=17, wait=1.0),
 FeaturedNote(pitch=7, wait=0.0),
 FeaturedNote(pitch=13, wait=0.96875),
 FeaturedNote(pitch=35, wait=0.05208333333333326),
 FeaturedNote(pitch=36, wait=0.28776041666666674),
 FeaturedNote(pitch=23, wait=0.17057291666666652),
 FeaturedNote(pitch=37, wait=0.0),
 FeaturedNote(pitch=20, wait=0.13411458333333348),
 FeaturedNote(pitch=38, wait=0.0),
 FeaturedNote(pitch=24, wait=0.0),
 FeaturedNote(pitch=4, wait=0.0),
 FeaturedNote(pitch=1

In [32]:
validation_sequences, validation_pitch, validation_wait = feature_songs(validation_songs)

In [20]:
train_sequences, train_pitch, train_wait = feature_songs(train_songs)

In [23]:
np.min(train_sequences)

0.0

In [22]:
train_pitch.shape

(5996034, 41)

In [24]:
model = Sequential()

# Recurrent layer
model.add(LSTM(64, input_shape=(nb_notes_history, nb_features),
               return_sequences=False, dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(nb_pitches, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
run_time = dt.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

In [26]:
run_time

'2019-06-30_00:19:50'

In [27]:
# Create callbacks
callbacks = [keras.callbacks.TensorBoard(log_dir='tb_logs/notebased_featuresv1_{}'.format(run_time), histogram_freq=0, write_graph=True, write_grads=False, 
                                         write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, 
                                         embeddings_data=None, update_freq='batch'),
             EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('models/notebased_featuresv1_{}.h5'.format(run_time), save_best_only=True, save_weights_only=False)]

In [28]:
train_sequences.shape

(5996034, 16, 42)

In [ ]:
history = model.fit(train_sequences,  train_pitch, 
                    batch_size=2048, epochs=5,
                    callbacks=callbacks,
                    validation_data=(validation_sequences, validation_pitch))

W0630 00:20:39.629843 140479974627072 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5996034 samples, validate on 660126 samples
Epoch 1/5
 899072/5996034 [===>..........................] - ETA: 14:37 - loss: 3.3973 - accuracy: 0.0891